In [62]:
baseUrl = "https://apl.unob.cz/StudijniProgramy/Studium/4/Kyberneticka-bezpecnost"

https://docs.aiohttp.org/en/stable/client_quickstart.html

In [63]:
import aiohttp

In [64]:
async with aiohttp.ClientSession() as session:
    async with session.get(baseUrl) as resp:
        print(resp.status)
        htmlData = await resp.text()

200


In [65]:
positionStart = htmlData.find('<table')
positionEnd = htmlData.find('</table')
print(positionStart, positionEnd)

2543 4042


In [66]:
tableData = htmlData[positionStart:positionEnd]
rows = tableData.split('<tr')
for row in rows:
    print(row.replace('\n', ''))

            ="table table-sm studijniProgramTable">
            </tr>d>istersk&#xFD;rogramu
            </tr>d>zen&#x10D;n&#xED;
            </tr>d>10D;e&#x161;tina
            </tr>d>etstudia
            </tr>d>ulta vojensk&#xFD;ch technologi&#xED;
            </tr>d>Uchazeči podávají přihlášku ke studiu ve studijním programu elektronickou formou na předepsaném formuláři. Požadavky na přijímací řízení jsou schvalovány akademickým senátem Fakulty vojenských technologií na příslušný akademický rok.</p><p>Přijímací zkouška standardně zahrnuje test studijních předpokladů.</p>


In [67]:
import re

"\/StudijniProgramy\/PredmetDetail\/([0-9]+)\/Algoritmizace-a-programovani"

'\\/StudijniProgramy\\/PredmetDetail\\/([0-9]+)\\/Algoritmizace-a-programovani'

In [68]:
htmlData[positionStart:positionEnd]

'<table class="table table-sm studijniProgramTable">\r\n            <tr>\r\n              <td>\r\n                Typ studijního programu\r\n              </td>\r\n              <td>\r\n                magistersk&#xFD;\r\n              </td>\r\n            </tr>\r\n            <tr>\r\n              <td>\r\n                Forma studia\r\n              </td>\r\n              <td>\r\n                prezen&#x10D;n&#xED;\r\n              </td>\r\n            </tr>\r\n            <tr>\r\n              <td>\r\n                Jazyk studia\r\n              </td>\r\n              <td>\r\n                &#x10D;e&#x161;tina\r\n              </td>\r\n            </tr>\r\n            <tr>\r\n              <td>\r\n                Doba studia\r\n              </td>\r\n              <td>\r\n                5 let\r\n              </td>\r\n            </tr>\r\n            <tr>\r\n              <td>\r\n                Garantuje\r\n              </td>\r\n              <td>\r\n                Fakulta vo

# Lekce II

In [69]:
def func(a):
    return a + 1

def funcB(a):
    return "Ahoj " + a

def logit(func):
    def newFunc(a):
        print(func, 'volání s parametrem', a)
        return func(a)
    return newFunc

b = logit(func)
b(2)
logit(func)(4)
logit(funcB)("Josef")

<function func at 0x7fac8a7b6440> volání s parametrem 2
<function func at 0x7fac8a7b6440> volání s parametrem 4
<function funcB at 0x7faca02f4310> volání s parametrem Josef


'Ahoj Josef'

In [70]:
@logit
def func(a):
    return a + 1

@logit
def funcB(a):
    return "Ahoj " + a

func(4)
funcB("Josef");

<function func at 0x7fac8a7b5c60> volání s parametrem 4
<function funcB at 0x7fac8a7b5870> volání s parametrem Josef


Pracujeme s daty stahovanými z internetu. Především ve fázi experimentování je velmi žádoucí stejná data nestahovat opakovaně. V případě synchronní funkce bychom mohli využít dekorátor @cache. Pro asynchronní funkci si vytvoříme podobnou funkci sami.

In [71]:
def singleCall(asyncFunc):
    awaitedResults = {} 
    async def resultFunc(singleParam):
        result = awaitedResults.get(singleParam, None)
        if result is None:
            result = await asyncFunc(singleParam)
            awaitedResults[singleParam] = result
        return result
    return resultFunc

Stahování http data "obalíme" do funkce.

In [72]:
import aiohttp

baseUrl = "https://apl.unob.cz/StudijniProgramy/Studium/4/Kyberneticka-bezpecnost"

@singleCall
async def getData(baseUrl): 
    async with aiohttp.ClientSession() as session:
        async with session.get(baseUrl) as resp:
            htmlData = await resp.text()
    return htmlData

V získané stránce najdeme všechny výskyty vzoru. Všimněte si struktury na výstupu (tuple).

In [73]:
import re

textData = await getData(baseUrl)

processor = re.compile('(\/StudijniProgramy\/PredmetDetail\/([0-9]+)\/([^\"]+))')
items = processor.findall(textData)
for item in items:
    print(item)

('/StudijniProgramy/PredmetDetail/3/Algoritmizace-a-programovani', '3', 'Algoritmizace-a-programovani')
('/StudijniProgramy/PredmetDetail/448/Anglicky-jazyk', '448', 'Anglicky-jazyk')
('/StudijniProgramy/PredmetDetail/327/Leadership', '327', 'Leadership')
('/StudijniProgramy/PredmetDetail/602/Matematika-I', '602', 'Matematika-I')
('/StudijniProgramy/PredmetDetail/664/Metodologie', '664', 'Metodologie')
('/StudijniProgramy/PredmetDetail/416/Telesna-vychova', '416', 'Telesna-vychova')
('/StudijniProgramy/PredmetDetail/673/Zaklady-prava', '673', 'Zaklady-prava')
('/StudijniProgramy/PredmetDetail/449/Anglicky-jazyk', '449', 'Anglicky-jazyk')
('/StudijniProgramy/PredmetDetail/597/Bezpecnost-a-obrana-statu', '597', 'Bezpecnost-a-obrana-statu')
('/StudijniProgramy/PredmetDetail/660/Cislicova-technika', '660', 'Cislicova-technika')
('/StudijniProgramy/PredmetDetail/4/Informatika', '4', 'Informatika')
('/StudijniProgramy/PredmetDetail/328/Leadership', '328', 'Leadership')
('/StudijniProgramy/Pr

Pro lepší použití vytvoříme funkci. 

Lze použít dekorátor @cache? Opatrně s tvrzením!

## Extrakce dat

In [74]:
def extractData(data, pattern):
    processor = re.compile(pattern)
    items = processor.findall(data)
    for item in items:
        yield item

Jednotlivé prvky tuple pojmenujeme.

In [75]:
def nameTuple(data, names):
    result = {}
    for key, value in zip(names, data):
        result[key] = value
    return result

def nameAllRows(data, names):
    for row in data:
        yield nameTuple(row, names)

baseUrl = "https://apl.unob.cz/StudijniProgramy/Studium/4/Kyberneticka-bezpecnost"
pattern = '(\/StudijniProgramy\/PredmetDetail\/([0-9]+)\/([^\"]+))'
names = ["url", "id", "name"]

data = await getData(baseUrl)
tuples = extractData(data, pattern)
jsons = nameAllRows(tuples, names)

for item in jsons:
    print(item)


{'url': '/StudijniProgramy/PredmetDetail/3/Algoritmizace-a-programovani', 'id': '3', 'name': 'Algoritmizace-a-programovani'}
{'url': '/StudijniProgramy/PredmetDetail/448/Anglicky-jazyk', 'id': '448', 'name': 'Anglicky-jazyk'}
{'url': '/StudijniProgramy/PredmetDetail/327/Leadership', 'id': '327', 'name': 'Leadership'}
{'url': '/StudijniProgramy/PredmetDetail/602/Matematika-I', 'id': '602', 'name': 'Matematika-I'}
{'url': '/StudijniProgramy/PredmetDetail/664/Metodologie', 'id': '664', 'name': 'Metodologie'}
{'url': '/StudijniProgramy/PredmetDetail/416/Telesna-vychova', 'id': '416', 'name': 'Telesna-vychova'}
{'url': '/StudijniProgramy/PredmetDetail/673/Zaklady-prava', 'id': '673', 'name': 'Zaklady-prava'}
{'url': '/StudijniProgramy/PredmetDetail/449/Anglicky-jazyk', 'id': '449', 'name': 'Anglicky-jazyk'}
{'url': '/StudijniProgramy/PredmetDetail/597/Bezpecnost-a-obrana-statu', 'id': '597', 'name': 'Bezpecnost-a-obrana-statu'}
{'url': '/StudijniProgramy/PredmetDetail/660/Cislicova-technika

In [76]:
def countUp(c):
    for index in range(c):
        print('CountUp', index)
        yield index
        
def addNumber(dataFlow, number):
    for row in dataFlow:
        print('addNumber', row)
        yield row + number
        
result = addNumber(countUp(10), 100)
print('going to transform into list')
print(list(result))

going to transform into list
CountUp 0
addNumber 0
CountUp 1
addNumber 1
CountUp 2
addNumber 2
CountUp 3
addNumber 3
CountUp 4
addNumber 4
CountUp 5
addNumber 5
CountUp 6
addNumber 6
CountUp 7
addNumber 7
CountUp 8
addNumber 8
CountUp 9
addNumber 9
[100, 101, 102, 103, 104, 105, 106, 107, 108, 109]


## Posloupnost zpracování

In [77]:
import asyncio

def modifyUrl(data, urlPrefix):
    for row in data:
        row['url'] = urlPrefix + row['url']
        yield row
    
async def getPages(data):
    result = []
    for row in data:
        text = await getData(row['url'])
        row['html'] = text[:100]
        result.append(row)
    return result
    
baseUrl = "https://apl.unob.cz/StudijniProgramy/Studium/4/Kyberneticka-bezpecnost"
pattern = '(\/StudijniProgramy\/PredmetDetail\/([0-9]+)\/([^\"]+))'
names = ["url", "id", "name"]

data = await getData(baseUrl)
tuples = extractData(data, pattern)
jsons = nameAllRows(tuples, names)
jsonsEx = modifyUrl(jsons, 'https://apl.unob.cz')
jsonsWithHTML = await getPages(jsonsEx)

for item in jsonsWithHTML:
    print(item)

{'url': 'https://apl.unob.cz/StudijniProgramy/PredmetDetail/3/Algoritmizace-a-programovani', 'id': '3', 'name': 'Algoritmizace-a-programovani', 'html': '\r\n\r\n<!DOCTYPE html>\r\n<html>\r\n<head>\r\n  <meta charset="utf-8" />\r\n  <meta name="viewport" content="wi'}
{'url': 'https://apl.unob.cz/StudijniProgramy/PredmetDetail/448/Anglicky-jazyk', 'id': '448', 'name': 'Anglicky-jazyk', 'html': '\r\n\r\n<!DOCTYPE html>\r\n<html>\r\n<head>\r\n  <meta charset="utf-8" />\r\n  <meta name="viewport" content="wi'}
{'url': 'https://apl.unob.cz/StudijniProgramy/PredmetDetail/327/Leadership', 'id': '327', 'name': 'Leadership', 'html': '\r\n\r\n<!DOCTYPE html>\r\n<html>\r\n<head>\r\n  <meta charset="utf-8" />\r\n  <meta name="viewport" content="wi'}
{'url': 'https://apl.unob.cz/StudijniProgramy/PredmetDetail/602/Matematika-I', 'id': '602', 'name': 'Matematika-I', 'html': '\r\n\r\n<!DOCTYPE html>\r\n<html>\r\n<head>\r\n  <meta charset="utf-8" />\r\n  <meta name="viewport" content="wi'}
{'url': 'htt

In [78]:
stm = '<td>Předmět profilujícího základu</td>[^<]+<td>([^<]+)</td>'
def createPattern(attributeName):
    return f'<td>{attributeName}</td>[^<]+<td>([^<]+)</td>'

patterns = {
    "name": "Název předmětu",
    "grant": "Garant",
    "department": "Katedra",
    "spec": "Předmět specializace",
    "base": "Předmět profilujícího základu",
    "teoretical": "Teoretický předmět PZ",
    "state": "Státní zkouška",
    "multiSemester": "Vícesemestrální předmět",
    "otherSchool": "Předmět jiné školy",
    "notMandatory": "Volitelnost",
    "evaluation": "Klasifikace",
    "credits": "Kredity",
    "semester": "Dop. roč./sem.",
    "weeks": "Počet týdnů"
}

def appendAttributes(data):
    for row in data:
        for key, value in patterns.items():
            occurence = re.findall(createPattern(value), row['html'])
            if len(occurence) == 0:
                attValue = None
            else:
                attValue = occurence[0]
            row[key] = attValue
        yield row
        
baseUrl = "https://apl.unob.cz/StudijniProgramy/Studium/4/Kyberneticka-bezpecnost"
pattern = '(\/StudijniProgramy\/PredmetDetail\/([0-9]+)\/([^\"]+))'
names = ["url", "id", "name"]

data = await getData(baseUrl)
tuples = extractData(data, pattern)
jsons = nameAllRows(tuples, names)
jsonsEx = modifyUrl(jsons, 'https://apl.unob.cz')
jsonsWithHTML = await getPages(jsonsEx)
jsonsWithAttributes = appendAttributes(jsonsWithHTML)

for item in jsonsWithAttributes:
    print(item)

{'url': 'https://apl.unob.cz/StudijniProgramy/PredmetDetail/3/Algoritmizace-a-programovani', 'id': '3', 'name': None, 'html': '\r\n\r\n<!DOCTYPE html>\r\n<html>\r\n<head>\r\n  <meta charset="utf-8" />\r\n  <meta name="viewport" content="wi', 'grant': None, 'department': None, 'spec': None, 'base': None, 'teoretical': None, 'state': None, 'multiSemester': None, 'otherSchool': None, 'notMandatory': None, 'evaluation': None, 'credits': None, 'semester': None, 'weeks': None}
{'url': 'https://apl.unob.cz/StudijniProgramy/PredmetDetail/448/Anglicky-jazyk', 'id': '448', 'name': None, 'html': '\r\n\r\n<!DOCTYPE html>\r\n<html>\r\n<head>\r\n  <meta charset="utf-8" />\r\n  <meta name="viewport" content="wi', 'grant': None, 'department': None, 'spec': None, 'base': None, 'teoretical': None, 'state': None, 'multiSemester': None, 'otherSchool': None, 'notMandatory': None, 'evaluation': None, 'credits': None, 'semester': None, 'weeks': None}
{'url': 'https://apl.unob.cz/StudijniProgramy/PredmetDetai

# Lekce III

In [79]:
async def makeItAwaitable(item):
    return item

In [80]:
async def asyncRange(count):
    for item in range(count):
        yield item

In [81]:
async def operation(data):
    async for row in data:
        yield row + 1

In [82]:
async def asyncList(data):
    result = []
    async for item in data:
        result.append(item)
    return result

In [83]:
async def asyncClose(data):
    async for item in data:
        continue

In [84]:
async def asyncPrint(data):
    async for item in data:
        print(item)
        yield item

In [85]:
await asyncClose(asyncPrint(operation(asyncRange(3))))

1
2
3


In [86]:
result = await asyncList(operation(asyncRange(3)))
print(result)

[1, 2, 3]


## Pokračování získávání dat

In [87]:
def singleCall(asyncFunc):
    awaitedResults = {} 
    async def resultFunc(singleParam):
        result = awaitedResults.get(singleParam, None)
        if result is None:
            result = await asyncFunc(singleParam)
            awaitedResults[singleParam] = result
        return result
    return resultFunc

In [88]:
import aiohttp

baseUrl = "https://apl.unob.cz/StudijniProgramy/Studium/4/Kyberneticka-bezpecnost"

@singleCall
async def getData(baseUrl): 
    async with aiohttp.ClientSession() as session:
        async with session.get(baseUrl) as resp:
            htmlData = await resp.text()
    return htmlData

In [89]:
async def extractData(data, pattern):
    processor = re.compile(pattern)
    items = processor.findall(data)
    for item in items:
        yield item

In [90]:
def nameTuple(data, names):
    result = {}
    for key, value in zip(names, data):
        result[key] = value
    return result

async def nameAllRows(data, names):
    async for row in data:
        yield nameTuple(row, names)

In [91]:
import asyncio

async def modifyUrl(data, urlPrefix):
    async for row in data:
        row['url'] = urlPrefix + row['url']
        yield row
    
async def getPages(data):
    async for row in data:
        text = await getData(row['url'])
        row['html'] = text
        yield row
    

In [92]:
def createPattern(attributeName):
    return f'<td>{attributeName}</td>[^<]+<td>([^<]+)</td>'

patterns = {
    "name": "Název předmětu",
    "grant": "Garant",
    "department": "Katedra",
    "spec": "Předmět specializace",
    "base": "Předmět profilujícího základu",
    "teoretical": "Teoretický předmět PZ",
    "state": "Státní zkouška",
    "multiSemester": "Vícesemestrální předmět",
    "otherSchool": "Předmět jiné školy",
    "notMandatory": "Volitelnost",
    "evaluation": "Klasifikace",
    "credits": "Kredity",
    "semester": "Dop. roč./sem.",
    "weeks": "Počet týdnů"
}

async def appendAttributes(data):
    async for row in data:
        for key, value in patterns.items():
            occurence = re.findall(createPattern(value), row['html'])
            if len(occurence) == 0:
                attValue = None
            else:
                attValue = occurence[0]
            row[key] = attValue
        yield row

In [93]:
async def removeAttribute(data, name):
    async for item in data:
        del item[name]
        yield item

In [94]:
async def asyncList(data):
    result = []
    async for item in data:
        result.append(item)
    return result

In [95]:
async def asyncClose(data):
    async for item in data:
        continue

In [128]:
import re
baseUrl = "https://apl.unob.cz/StudijniProgramy/Studium/4/Kyberneticka-bezpecnost"
pattern = '(\/StudijniProgramy\/PredmetDetail\/([0-9]+)\/([^\"]+))'
names = ["url", "id", "name"]

data = await getData(baseUrl)
tuples = extractData(data, pattern)
jsons = nameAllRows(tuples, names)
jsonsEx = modifyUrl(jsons, 'https://apl.unob.cz')
jsonsWithHTML = getPages(jsonsEx)
jsonsAttributed = appendAttributes(jsonsWithHTML)
jsonsWOHtml = removeAttribute(jsonsAttributed, 'html')
#printed = asyncPrint(jsonsWOHtml)
allRows = await asyncList(jsonsWOHtml)

In [ ]:
import pandas as pd
pd.DataFrame(allRows)

,url,id,name,grant,department,spec,base,teoretical,state,multiSemester,otherSchool,notMandatory,evaluation,credits,semester,weeks
0,https://apl.unob.cz/StudijniProgramy/PredmetDe...,3,Algoritmizace a programov&#xE1;n&#xED; (AlgProg),None,Katedra informatiky a kybernetick&#xFD;ch oper...,NE,ANO,NE,NE,\r\nNE,NE,Povinn&#xFD;,Z&#xE1;po&#x10D;et &#x2B; Zkou&#x161;ka,6,1/1,0
1,https://apl.unob.cz/StudijniProgramy/PredmetDe...,448,Anglick&#xFD; jazyk (AJ),None,Centrum jazykov&#xE9;ho vzd&#x11B;l&#xE1;v&#xE...,NE,NE,NE,NE,None,NE,Povinn&#xFD;,Z&#xE1;po&#x10D;et,2,1/1,12
2,https://apl.unob.cz/StudijniProgramy/PredmetDe...,327,Leadership (LEA),None,Katedra leadershipu,NE,NE,NE,NE,None,NE,Povinn&#xFD;,Z&#xE1;po&#x10D;et,2,1/1,12
3,https://apl.unob.cz/StudijniProgramy/PredmetDe...,602,Matematika I (MAT I),None,Katedra matematiky a fyziky,NE,ANO,NE,NE,\r\nNE,NE,Povinn&#xFD;,Z&#xE1;po&#x10D;et &#x2B; Zkou&#x161;ka,6,1/1,12
4,https://apl.unob.cz/StudijniProgramy/PredmetDe...,664,Metodologie (MET),None,Centrum bezpe&#x10D;nostn&#xED;ch a vojenskost...,NE,ANO,NE,NE,\r\nNE,NE,Povinn&#xFD;,Klasifikovan&#xFD; z&#xE1;po&#x10D;et,6,1/1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,https://apl.unob.cz/StudijniProgramy/PredmetDe...,425,T&#x11B;lesn&#xE1; v&#xFD;chova (TV),None,Centrum t&#x11B;lesn&#xE9; v&#xFD;chovy a sportu,NE,NE,NE,NE,None,NE,Povinn&#xFD;,Z&#xE1;po&#x10D;et,2,5/10,12
77,https://apl.unob.cz/StudijniProgramy/PredmetDe...,676,Zpracov&#xE1;n&#xED; diplomov&#xE9; pr&#xE1;ce...,None,Katedra informatiky a kybernetick&#xFD;ch oper...,NE,NE,NE,NE,\r\nNE,NE,Povinn&#xFD;,Z&#xE1;po&#x10D;et,16,5/10,12
78,https://apl.unob.cz/StudijniProgramy/PredmetDe...,65888,Francouzsk&#xFD; jazyk (FJ),None,Centrum jazykov&#xE9;ho vzd&#x11B;l&#xE1;v&#xE...,NE,NE,NE,NE,None,NE,Povinn&#x11B; voliteln&#xFD;,Z&#xE1;po&#x10D;et,2,5/10,12
79,https://apl.unob.cz/StudijniProgramy/PredmetDe...,65892,N&#x11B;meck&#xFD; jazyk (NJ),None,Centrum jazykov&#xE9;ho vzd&#x11B;l&#xE1;v&#xE...,NE,NE,NE,NE,None,NE,Povinn&#x11B; voliteln&#xFD;,Z&#xE1;po&#x10D;et,2,5/10,12


Podmínky pro sběr dat:
- Data musí být homogenní.
- Jestliže je va datech atribut ve tvaru vektoru, listu, dictionary, tak je nelze uložit
- Musí být typu: string, int, (uuid), datetime, bool
Jak postupovat při plnění databáze získanými daty?
- 

In [144]:
#ocisteni dat
import uuid
departments = {}
for row in allRows:
    depName = row.get('department', None)
    if depName is None:
        continue
    depId = departments.get(depName, None)
    if depId is None:
        depId = f'{uuid.uuid1()}'
        departments[depName] = depId
    row['department_id'] = depId

print(departments)
departmentList = []
for key, value in departments.items():
    departmentList.append({'id': value, 'name': key})
departmentList
        


{'Katedra informatiky a kybernetick&#xFD;ch operac&#xED;': '78a1355c-6be0-11ed-9e9b-0242ac120003', 'Centrum jazykov&#xE9;ho vzd&#x11B;l&#xE1;v&#xE1;n&#xED;': '78a13660-6be0-11ed-9e9b-0242ac120003', 'Katedra leadershipu': '78a136b0-6be0-11ed-9e9b-0242ac120003', 'Katedra matematiky a fyziky': '78a136ec-6be0-11ed-9e9b-0242ac120003', 'Centrum bezpe&#x10D;nostn&#xED;ch a vojenskostrategick&#xFD;ch studi&#xED;': '78a13728-6be0-11ed-9e9b-0242ac120003', 'Centrum t&#x11B;lesn&#xE9; v&#xFD;chovy a sportu': '78a1375a-6be0-11ed-9e9b-0242ac120003', 'Katedra &#x159;&#xED;zen&#xED; zdroj&#x16F;': '78a1378c-6be0-11ed-9e9b-0242ac120003', 'Katedra elektrotechniky': '78a137c8-6be0-11ed-9e9b-0242ac120003', 'Katedra teorie vojenstv&#xED;': '78a1384a-6be0-11ed-9e9b-0242ac120003', 'Katedra komunika&#x10D;n&#xED;ch technologi&#xED;, elektronick&#xE9;ho boje a radiolokace': '78a138e0-6be0-11ed-9e9b-0242ac120003'}


[{'id': '78a1355c-6be0-11ed-9e9b-0242ac120003',
  'name': 'Katedra informatiky a kybernetick&#xFD;ch operac&#xED;'},
 {'id': '78a13660-6be0-11ed-9e9b-0242ac120003',
  'name': 'Centrum jazykov&#xE9;ho vzd&#x11B;l&#xE1;v&#xE1;n&#xED;'},
 {'id': '78a136b0-6be0-11ed-9e9b-0242ac120003', 'name': 'Katedra leadershipu'},
 {'id': '78a136ec-6be0-11ed-9e9b-0242ac120003',
  'name': 'Katedra matematiky a fyziky'},
 {'id': '78a13728-6be0-11ed-9e9b-0242ac120003',
  'name': 'Centrum bezpe&#x10D;nostn&#xED;ch a vojenskostrategick&#xFD;ch studi&#xED;'},
 {'id': '78a1375a-6be0-11ed-9e9b-0242ac120003',
  'name': 'Centrum t&#x11B;lesn&#xE9; v&#xFD;chovy a sportu'},
 {'id': '78a1378c-6be0-11ed-9e9b-0242ac120003',
  'name': 'Katedra &#x159;&#xED;zen&#xED; zdroj&#x16F;'},
 {'id': '78a137c8-6be0-11ed-9e9b-0242ac120003',
  'name': 'Katedra elektrotechniky'},
 {'id': '78a1384a-6be0-11ed-9e9b-0242ac120003',
  'name': 'Katedra teorie vojenstv&#xED;'},
 {'id': '78a138e0-6be0-11ed-9e9b-0242ac120003',
  'name': 'Kate

vytvoření strukutry pomocí dictionary
tím že použiju dictionary v dicrtionary, tak zkrátím dobu hledání(díky indexům - key), procházím key, ne values 

In [ ]:
db = {
    'users':{
        1: {},
        '1':{},
    },
    'groups':{
        1:{},
        '1':{},
    }    
} 

In [97]:
async def makeItAwaitable(value):
    return value

In [98]:
b = lambda x: makeItAwaitable(x)
await b(3)

3

# Lekce IV

## Generátory

### Motivace

In [99]:
class StateMachineClass:
    def __init__ (self, params):
        self.params = params
        self.state = 0

    def __iter__ (self):
        return self

    def __next__ (self):
        self.state += 1
        if self.state == 1:
            return 'A'
        elif self.state == 2:
            return 'hoj'
        elif self.state == 3:
            return ' '
        elif self.state == 4:
            return self.params
        else:
            raise StopIteration()
        return None
    

In [100]:
stateMachine = StateMachineClass('Pepo')
while True:
    try:
        result = stateMachine.__next__()
        print(result)
    except StopIteration:
        break

A
hoj
 
Pepo


In [101]:
stateMachine = StateMachineClass('Pepo')
for item in stateMachine:
    print(item)

A
hoj
 
Pepo


In [102]:
def StateMachineFunc(params):
    yield 'A'
    yield 'hoj'
    yield ' '
    yield params
    pass

stateMachine = StateMachineFunc('Pepo')
for item in stateMachine:
    print(item)

A
hoj
 
Pepo


### Principy

In [103]:
def StateMachineFuncEx(params):
    print('A', 'will be yielded')
    yield 'A'
    print('A', 'has been yielded')
    print('hoj', 'will be yielded')
    yield 'hoj'
    print('hoj', 'has been yielded')
    print(' ', 'will be yielded')
    yield ' '
    print(' ', 'has been yielded')
    print(params, 'will be yielded')
    yield params
    print(params, 'has been yielded')
    pass

for item in StateMachineFuncEx('Pepo'):
    print(item)
    print('# next iteration')
    

A will be yielded
A
# next iteration
A has been yielded
hoj will be yielded
hoj
# next iteration
hoj has been yielded
  will be yielded
 
# next iteration
  has been yielded
Pepo will be yielded
Pepo
# next iteration
Pepo has been yielded


In [104]:
def myRange(start=0, step=1, stop=10):
    current = start
    while True:
        print('going to yield', current)
        yield current
        current = current + step
        if current >= stop:
            break

In [105]:
def printAll(data, comment=""):
    for item in data:
        print(comment, item)
        yield item

In [106]:
def close(data):
    for item in data:
        continue

In [107]:
gen1 = myRange(stop=10)
gen1 = printAll(gen1, '=>')
gen1 = close(gen1)

going to yield 0
=> 0
going to yield 1
=> 1
going to yield 2
=> 2
going to yield 3
=> 3
going to yield 4
=> 4
going to yield 5
=> 5
going to yield 6
=> 6
going to yield 7
=> 7
going to yield 8
=> 8
going to yield 9
=> 9


### Zřetězené generátory 

In [108]:
def myRange(start=0, step=1, stop=10):
    current = start
    while True:
        yield current
        current = current + step
        if current == stop:
            break

In [109]:
def printAll(data, comment=""):
    for item in data:
        print(comment, item)
        yield item

In [110]:
def limit(data, limit):
    for index, item in enumerate(data):
        yield item
        if index >= limit:
            break

In [111]:
def skip(data, skip):
    for index, item in enumerate(data):
        if index >= skip:
            break
    for item in data:
        yield item

In [112]:
def close(data):
    for item in data:
        continue

In [113]:
gen1 = myRange()
gen1 = printAll(gen1, 'wo limits')
gen1 = skip(gen1, 10)
gen1 = limit(gen1, 10)
gen1 = printAll(gen1, ':)')
ge1 = close(gen1)

wo limits 0
wo limits 1
wo limits 2
wo limits 3
wo limits 4
wo limits 5
wo limits 6
wo limits 7
wo limits 8
wo limits 9


### Conversion :))

In [114]:
def asStreamOperator(op):
    def result(data):
        for item in data:
            yield op(item)
    return result

In [115]:
@asStreamOperator
def incBy100(item):
    return item + 100

In [116]:
gen1 = myRange(stop=10)
gen1 = incBy100(gen1)
gen1 = printAll(gen1)
gen1 = close(gen1)

 100
 101
 102
 103
 104
 105
 106
 107
 108
 109


# Lekce V

## Otázky

In [117]:
async def asyncPrint(data):
    async for item in data:
        print(item)
        yield item

In [118]:
async def asyncClose(data):
    async for item in data:
        continue

In [119]:
async def asyncRange(start=0, step=1, stop=10):
    current = start
    while True:
        yield current
        current = current + step
        if current == stop:
            break

In [120]:
data = await asyncClose(asyncPrint(asyncRange()))

0
1
2
3
4
5
6
7
8
9


In [121]:
g = asyncRange()
await g.asend(None)

0

In [122]:
class agWrapper():
    def __init__(self, asyncGenObj):
        self.asyncGen = asyncGenObj
        
    def __iter__(self):
        return self
    
    async def __next__(self):
        hasResult = True
        try:
            result = (await self.asyncGen.asend(None))
        except StopAsyncIteration:
            hasResult = False
        if not hasResult:
            raise StopIteration
        return result

In [123]:
g = agWrapper(asyncPrint(asyncRange()))
for item in (g):
    await item
    continue
    #print(index, await item)
    

0
1
2
3
4
5
6
7
8
9


RuntimeError: coroutine raised StopIteration

In [ ]:
def toSyncGenOfAwaitables(asyncGen):
    while True:
        try:
            result = asyncGen.__anext__()
            yield result
        except StopAsyncIteration:
            break

In [ ]:
def asSyncGenOfAwaitables(asyncGenFuntion):
    def resultFunc(asyncGen):
        for item in toSyncGenOfAwaitables(asyncGen):
            yield item
        pass
    return resultFunc

In [ ]:
import inspect
def asSyncGenOfAwaitables(asyncGenFuntion):
      
    return resultFunc

In [ ]:
@asSyncGenOfAwaitables
async def asyncRange(start=0, step=1, stop=10):
    current = start
    while True:
        yield current
        current = current + step
        if current == stop:
            break

In [ ]:
@asSyncGenOfAwaitables
async def asyncPrint(data):
    for item in data:
        itemValue = await item
        print(itemValue)
        yield itemValue

In [ ]:
@asSyncGenOfAwaitables
async def asyncClose(data):
    for item in data:
        continue

In [ ]:
print(asyncClose(asyncPrint(asyncRange())))
print('###########')
for item in asyncClose(asyncPrint(asyncRange())):
    itemValue = await item
    #print(itemValue)

In [ ]:
data = asyncClose(asyncPrint(asyncRange()))
for item in data:
    itemValue = await item